# Network diagram

In [ ]:
#importing packages 
import pandas as pd
import numpy as np

In [ ]:
df_between = pd.read_csv('')
df_customers = pd.read_csv('')

Before we move foward I want to check to see if the accounts that are being sent money are Lloyds customers or not.

In [ ]:
unique_between = set(df_between['to_account'].unique()) #Turning these arrays into sets allow us to use the .issubset method
customers = set(np.array(df_customers.index)) 
unique_between.issubset(customers) #This will return true if all the customers being sent money are also Lloyds customers

: 

So not every to_account in the between df is a lloyds customer (or a customer we have data to). 
Lets add a column to (binary) to whether the to_account is a lloyds customer or not.

In [ ]:
unique_between = df_between['to_account'].unique() #Gets an array of all customer ID's being sent money
customers = (np.array(df_customers.index)) #Gets an array of all the Customer ID's we have data to
dict = {} # Initialising my dictionary
for i in range(0,len(unique_between)):
    dict[unique_between[i]] = unique_between[i] in customers #This will create a dictionary where every key is an element in unique_between and the corresponding value is T/F depending on if that customer is a Lloyds customer
for i in range(0,len(customers)):
    dict[customers[i]] = True
df_between['to_lloyds_customer'] = df_between['to_account'].map(lambda x: dict[x]) #We use this dictionary to map a new column
df_between.head()


: 

In [ ]:
print('We have', len(df_between[(df_between.to_lloyds_customer == 0)]), 'out of', len(df_between), '(',round(100*len(df_between[(df_between.to_lloyds_customer == 0)])/len(df_between)), '%) transactions going to non Lloyds customers')

: 

10% of all transactions is substantial, so we cannot remove these from the diagram

Now let's create a special df for this diagram, to do this we add up all transactions between eachother, so we will lose date and transaction ID

In [ ]:
network_diagram = df_between.groupby(['from_account', 'to_account']).agg({'money_amount': 'sum', 'to_lloyds_customer': pd.Series.mode}) #We want one row for each unique combination of from and to accounts, so we groupby from and to accounts and sum money.

: 

In [ ]:
network_diagram_sample = network_diagram.sample(10000, random_state=1)
network_diagram_sample = network_diagram_sample.reset_index()
network_diagram_sample.head()

: 

In [ ]:
edges = pd.DataFrame()
edges['Source'] = network_diagram_sample['from_account'].astype(int)
edges['Target'] = network_diagram_sample['to_account'].astype(int)
edges['Type'] = 'Undirected'
edges['Weight'] = network_diagram_sample['money_amount']
edges.head()


: 

Exporting the data to load into gephi

In [ ]:
#edges.to_csv('/Users/sasha/Desktop/DATASCIENCE/2023/Mini-Project/data.nosync/networkdiagram/pregephi_edges.csv')



: 

Bringing the exported data from gephi back

In [ ]:
import json

f = open('/Users/sasha/Desktop/DATASCIENCE/2023/Mini-Project/data.nosync/networkdiagram/network_tableau.json')

json_data = json.load(f)
temp = [[json_data['nodes'][i]['key'], json_data['nodes'][i]['attributes']['x'], json_data['nodes'][i]['attributes']['y'], json_data['nodes'][i]['attributes']['size']] for i in range(0,len(json_data['nodes']))]
nodes = pd.DataFrame(np.array(temp), columns = ['ID', 'X', 'Y', 'Size'])
nodes['Lloyds_Customer'] = nodes['ID'].apply(lambda x: dict[x])


: 

### The code below takes too long, won't scale

In [ ]:
edges['X_target'] = edges['Target'].apply(lambda x: nodes[(nodes.ID == str(x))]['X'].sum())
edges['Y_target'] = edges['Target'].apply(lambda x: nodes[(nodes.ID == str(x))]['Y'].sum())

: 

In [ ]:
edges['Direction'] = edges['Source'].astype(str) + ' -> ' + edges['Target'].astype(str)
edgescopy = edges.copy()
edgescopy['Source'] = edges['Target']
edgescopy['Target'] = edges['Source']
edges['Base'] = 1
edgescopy['Base'] = 2
edges = pd.concat([edges, edgescopy])

: 

In [ ]:
#nodes.to_csv('/Users/sasha/Desktop/DATASCIENCE/2023/Mini-Project/data.nosync/networkdiagram/nodes_XY.csv')
#edges.to_csv('/Users/sasha/Desktop/DATASCIENCE/2023/Mini-Project/data.nosync/networkdiagram/edges_XY.csv')

: 